In [0]:
#!pip install tensorflow==1.0                #Restart Runtime

In [0]:
import tensorflow as tf

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

In [0]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784               #input_width*input_height

n_Conv1 = 32
conv1_k = 5
stride_Conv1 = 1
max_pool1_k = 2

n_Conv2 = 64
conv2_k = 5
stride_Conv2 = 1
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_height//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) * n_Conv2

In [0]:
weights = {
    
    "wc1" : tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channels,n_Conv1])),
    "wc2" : tf.Variable(tf.random_normal([conv2_k,conv2_k,n_Conv1,n_Conv2])),
    "wh1" : tf.Variable(tf.random_normal([input_size_to_hidden,n_hidden])),
    "wo" : tf.Variable(tf.random_normal([n_hidden,n_out]))
}

biases = {
    
    "bc1" : tf.Variable(tf.random_normal([n_Conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_Conv2])),
    "bh1" : tf.Variable(tf.random_normal([n_hidden])),
    "bo" : tf.Variable(tf.random_normal([n_out]))
}

In [0]:
def conv(x,weights,biases,stride=1):
  out = tf.nn.conv2d(x,weights,padding="SAME",strides=[1,stride,stride,1])
  out = tf.nn.bias_add(out,biases)
  out = tf.nn.relu(out)

  return out

In [0]:
def maxpooling(x,k=2):
  return tf.nn.max_pool(x,padding="SAME",ksize=[1,k,k,1],strides=[1,k,k,1])

In [0]:
def cnn(x,weights,biases,keep_prob):
  x = tf.reshape(x,shape=[-1,input_height,input_width,input_channels])

  conv1 = conv(x,weights['wc1'],biases['bc1'],stride_Conv1)
  conv1_pool = maxpooling(conv1,max_pool1_k)

  conv2 = conv(conv1_pool,weights['wc2'],biases['bc2'],stride_Conv2)
  conv2_pool = maxpooling(conv2,max_pool2_k)

  hidden_input = tf.reshape(conv2_pool,shape=[-1,input_size_to_hidden])

  hidden_out_bef_act = tf.matmul(hidden_input,weights['wh1']) + biases['bh1']
  hidden_out_bef_dropout = tf.nn.relu(hidden_out_bef_act)

  hidden_out = tf.nn.dropout(hidden_out_bef_dropout,keep_prob)

  output = tf.matmul(hidden_out,weights['wo']) + biases['bo']

  return output

In [0]:
x = tf.placeholder("float",[None,input_pixels])
y = tf.placeholder(tf.int32,[None,n_out])
keep_prob = tf.placeholder("float")

In [0]:
pred = cnn(x,weights,biases,keep_prob)

In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))

In [0]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [17]:
batch_size = 100
for i in range(25):                   #NumIterations
  num_batches = int(mnist.train.num_examples/batch_size)
  total_cost = 0
  for j in range(num_batches):
    batch_x,batch_y = mnist.train.next_batch(batch_size)
    c , _ = sess.run([cost,optimize],feed_dict={x:batch_x,y:batch_y,keep_prob:0.8})
    total_cost += c
  print(int(total_cost), end=' ')

1064997 44464 21849 15021 11195 9601 9363 7343 7727 7454 6410 6768 4984 5090 4958 4083 4318 3645 4420 3317 3131 2981 2854 2647 2746 

In [18]:
predictions = tf.argmax(pred,1)
correct_labels = tf.argmax(y,1)

correct_predictions = tf.equal(predictions,correct_labels)

predictions, correct_predictions = sess.run([predictions,correct_predictions],feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})

print("Accuracy =", correct_predictions.sum()*100/mnist.test.num_examples,"%")

Accuracy = 98.6 %
